In [2]:
import numpy as np
import scipy.constants as const
import uncertainties as unc
import uncertainties.unumpy as up
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
def model_fit_and_plot(xdata, ydata, model, parameters, plot=False, resid=True, raw=True, preview=-1):
    '''Fit a line or curve, and plot/show the fit results.
    The function returns a parameters object with the fit parameters.
    '''

    my_params = parameters
    my_model = model
    model_fit=my_model.fit(ydata, params=my_params, x=xdata)

    if plot:
        if resid:
            # Plot with residuals side by side
            fig, (ax1, ax2) = plt.subplots(2, figsize=(16,9), gridspec_kw={'height_ratios':[2,1]})
            ax1.plot(xdata[:preview], model_fit.best_fit[:preview], '-', label='fit')
            if raw:
                ax1.plot(xdata, ydata, 'o', markersize=1, label='raw data')
            ax1.legend()
            ax1.set_title('Fitted data')
            ax2.plot(xdata[:preview], model_fit.residual[:preview], 'o', markersize=1)
            ax2.set_title('Residuals')
        else:
            # Plot just the fit, no residuals
            plt.plot(xdata, model_fit.best_fit, '-', label='fit')
            plt.plot(xdata, ydata, 'o', markersize=1,label='raw data')
            plt.legend()
            plt.title('Fitted data')
        

    return model_fit.params

In [3]:
# File opener function to extract data to an array
def fileOpener(file_name, data_columns, file_name2 = None, data_columns2 = ()):
    def columnExtractor(file_name, data_columns):
        def generatorHelper(file):
            for line in file:
                splitted = line.split('\t')
                yield splitted[data_columns[0]]
                yield splitted[data_columns[1]]

        with open(file_name, 'r') as file:
            print(f"Opening file: {file_name}")
            generator = generatorHelper(file)
            result = np.fromiter(generator, dtype = float)
            print(f"Closing file: {file_name}")

        return result.reshape((-1, 2))

    rawdata = columnExtractor(file_name, data_columns)

    if file_name2:
        rawdata2 = columnExtractor(file_name2, data_columns2 if len(data_columns2) != 0 else data_columns)

        if len(rawdata2) > len(rawdata):
            rawdata2 = rawdata2[-len(rawdata):]
        elif len(rawdata) > len(rawdata2):
            rawdata = rawdata[-len(rawdata2):]

        rawdata = np.column_stack((rawdata, rawdata2))

    print(rawdata.shape)
    return rawdata

In [4]:
def sine_sq_fit(file_num, path, params_est, plot=False):
    """Fits a series of Colim_[number].lvm files to a sine squared function. Returns a list containing the respective fitting parameter objects.

    **Arguments**
    file_num: list which contains integers corresponding to the files we want to fit
    path: RAW string containing user path to folder containing Colim_[number].lvm files
    params_est: list of size 4 containing integer or float fitting parameter estimates for offset, amplitude, frequency, and phase respectively
    plot: determines whether to plot each raw data and fit
    """

    # Creating model and parameters for fitting
    mod = ExpressionModel('off + amp * (sin(x*freq + phase))**2')
    params = mod.make_params(off=params_est[0], amp=params_est[1], freq=params_est[2], phase=params_est[3])
    fit_list = []

    # Fitting each file
    for num in file_num:
        file_name = path + "\Colim_" + str(num) + ".lvm"
        data = fileOpener(file_name, [0,3])
        x_data = data[:, 0]
        y_data = data[:, 1]
        my_fit = model_fit_and_plot(x_data, y_data, mod, params, plot=plot)
        fit_list.append(my_fit)

    return fit_list

In [2]:
def demod_fit_single(file_num, path, freq,sample_rate=200, front_cut=100, back_cut=100):
    """Fits a single Colim_[number].lvm file using demodulate at a specified frequency. Returns 
    
    **Arguments**
    file_num: list which contains integers corresponding to the files we want to fit
    path: RAW string containing user path to folder containing Colim_[number].lvm files
    freq: frequency at which to demodulate
    """

    file_name=path + "\Colim_" + str(num) + ".lvm"
    data=fileOpener(file_name, [0,3])
    cut_data = cutData(data, front_cut, sample_rate)
    amp1,phase1=cut_data[0].demodulate(freq, stride=0.005)
    fit1 = [amp1, phase1]
    amp2,phase2=cut_data[1].demodulate(freq,stride=0.005)
    fit2 = [amp2, phase2]

    return fit1, fit2